In [137]:
import pandas as pd
import numpy as np
from datetime import datetime
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [138]:
# Replace 'opensky_departures.csv' with your actual CSV filename
df = pd.read_csv("E:/Code/Hacklytics/Data/September/New/T_ONTIME_MARKETING.csv", low_memory=False, encoding='utf-8')

# Quick look at the data
display(df.head())

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,MKT_UNIQUE_CARRIER,BRANDED_CODE_SHARE,MKT_CARRIER_AIRLINE_ID,MKT_CARRIER,...,DIV1_AIRPORT,DIV1_AIRPORT_ID,DIV2_AIRPORT,DIV2_AIRPORT_ID,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV5_AIRPORT,DIV5_AIRPORT_ID
0,2024,3,9,1,7,9/1/2024 12:00:00 AM,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024,3,9,1,7,9/1/2024 12:00:00 AM,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024,3,9,1,7,9/1/2024 12:00:00 AM,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024,3,9,1,7,9/1/2024 12:00:00 AM,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024,3,9,1,7,9/1/2024 12:00:00 AM,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
df['FL_DATE'] = df['FL_DATE'].astype(str)

In [140]:
# Convert FlightDate to datetime
df['FlightDate'] = pd.to_datetime(df['FL_DATE'], format='%m/%d/%Y', errors='coerce')

# Convert scheduled times (CRSDepTime and CRSArrTime) from HHMM to hour (and minute) numeric values.
# For example, if CRSDepTime is 1230, convert it to 12.5 (i.e. 12 + 30/60).
def hhmm_to_float(hhmm):
    try:
        hhmm = int(hhmm)
        hours = hhmm // 100
        minutes = hhmm % 100
        return hours + minutes / 60.0
    except:
        return np.nan

df['DepHour'] = df['CRS_DEP_TIME'].apply(hhmm_to_float)
df['ArrHour'] = df['CRS_ARR_TIME'].apply(hhmm_to_float)

# Derive day-of-week from FlightDate (Monday=0, Sunday=6)
df['DayOfWeek'] = df['FlightDate'].dt.dayofweek

# Drop rows with missing scheduled time or other critical fields
df = df.dropna(subset=['OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DepHour', 'ArrHour', 'CRS_ELAPSED_TIME', 'DISTANCE'])

In [141]:
display(df.head())

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,MKT_UNIQUE_CARRIER,BRANDED_CODE_SHARE,MKT_CARRIER_AIRLINE_ID,MKT_CARRIER,...,DIV3_AIRPORT,DIV3_AIRPORT_ID,DIV4_AIRPORT,DIV4_AIRPORT_ID,DIV5_AIRPORT,DIV5_AIRPORT_ID,FlightDate,DepHour,ArrHour,DayOfWeek
0,2024,3,9,1,7,9/1/2024 12:00:00 AM,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,6.000000,8.933333,NaN
1,2024,3,9,1,7,9/1/2024 12:00:00 AM,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,21.333333,5.966667,NaN
2,2024,3,9,1,7,9/1/2024 12:00:00 AM,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,19.716667,20.733333,NaN
3,2024,3,9,1,7,9/1/2024 12:00:00 AM,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,18.166667,19.050000,NaN
4,2024,3,9,1,7,9/1/2024 12:00:00 AM,AA,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,9.000000,14.383333,NaN


In [142]:
# Define a function to compute the flight score:
def compute_flight_score(row):
    # If cancelled or diverted, score is 1 (worst)
    if row['CANCELLED'] == 1 or row['DIVERTED'] == 1:
        return 1.0
    # Sum delays (if delay is NaN, assume 0)
    dep_delay = row['DEP_DELAY'] if not pd.isnull(row['DEP_DELAY']) else 0
    arr_delay = row['ARR_DELAY'] if not pd.isnull(row['ARR_DELAY']) else 0
    total_delay = dep_delay + arr_delay
    # For every 10 minutes of delay, subtract 1 point from 10.
    score = 10 - total_delay / 10.0
    # Clip the score between 1 and 10.
    return max(1.0, min(10.0, score))

df['FlightScore'] = df.apply(compute_flight_score, axis=1)

In [143]:
flight_number = df['OP_CARRIER_FL_NUM']  # Save before feature selection

features = ['OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'DepHour', 'ArrHour', 'CRS_ELAPSED_TIME', 'DISTANCE', 'DayOfWeek']
target = 'FlightScore'

In [144]:
# We'll use LabelEncoder on UniqueCarrier, Origin, and Dest.
label_encoders = {}

for col in ['OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST']:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col].astype(str))

X = df[features]
y = df[target]

In [145]:
X_train, X_test, y_train, y_test, flight_num_train, flight_num_test = train_test_split(
    X, y, flight_number, test_size=0.2, random_state=42
)

In [146]:
model = xgb.XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    random_state=42
)

model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [147]:
# 9. Evaluate the Model
y_pred = model.predict(X_test)
y_pred_rounded = np.clip(np.round(y_pred, 1), 1, 10)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Test RMSE: {rmse:.2f}")


Test RMSE: 2.53


In [148]:
results = X_test.copy()
results['OP_CARRIER_FL_NUM'] = flight_num_test.values  # Add the tail numbers
results['ActualScore'] = y_test.values
results['PredictedScore'] = y_pred_rounded
results['Error'] = results['PredictedScore'] - results['ActualScore']
display(results.head(10))

,OP_UNIQUE_CARRIER,ORIGIN,DEST,DepHour,ArrHour,CRS_ELAPSED_TIME,DISTANCE,DayOfWeek,OP_CARRIER_FL_NUM,ActualScore,PredictedScore,Error
465761,6,206,99,6.333333,9.016667,161.0,957.0,NaN,2529,10.0,9.2,-0.8
360935,11,289,183,18.666667,20.033333,82.0,258.0,NaN,2908,10.0,7.7,-2.3
445053,11,161,185,12.383333,14.000000,217.0,1379.0,NaN,461,10.0,9.0,-1.0
262507,13,89,91,19.166667,19.533333,82.0,292.0,NaN,5112,1.0,8.6,7.6
155110,16,183,240,23.983333,5.633333,219.0,1514.0,NaN,2102,10.0,9.2,-0.8
208279,20,240,77,14.433333,15.850000,85.0,315.0,NaN,6116,10.0,9.0,-1.0
556990,17,183,289,9.500000,10.750000,75.0,258.0,NaN,3349,6.0,8.3,2.3
331245,1,252,287,8.433333,11.250000,169.0,993.0,NaN,2365,10.0,9.1,-0.9
429989,17,183,152,17.916667,23.000000,185.0,1235.0,NaN,3820,10.0,8.1,-1.9
376151,5,228,22,10.166667,13.666667,150.0,907.0,NaN,1046,8.5,9.4,0.9


In [149]:
# Revert encoded columns before saving
for col in ['OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST']:
    results[col] = label_encoders[col].inverse_transform(results[col])

results.to_csv("E:/Code/Hacklytics/Data/September/New/flight_score_predictions_sept.csv", index=False)